# 使用 ReAct 搜尋維基百科


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


這個筆記本是使用 Google `gemini-pro` 模型對 [ReAct: 整合語言模型推理和動作](https://arxiv.org/abs/2210.03629) 進行的最小化實作。你將會使用 ReAct 提示來組態一個模型在維基百科中搜尋，來尋找使用者問題的答案。


在此演練中，你將會學習如何執行以下操作：


1.   設定你的開發環境和 API 存取權限以使用 Gemini。
2.   使用 ReAct 少數範例提示。
3.   將新提示的模型用於多回合對話 (聊天)。
4.   將模型連接到 **Wikipedia API** 。
5.   與模型對話 (試試提出像是「艾菲爾鐵塔有多高？」之類的問題)，並觀看模型搜尋 Wikipedia。


> 備註: 此頁面的非原始程式碼材料經採用創作共用許可 - 署名-相同方式分享 ( CC-BY-SA 4.0 ), https://creativecommons.org/licenses/by-sa/4.0/legalcode.


### 背景


[ReAct](https://arxiv.org/abs/2210.03629) 是一種提示方法，允許語言模型追蹤其思考過程和回答使用者問題所需的步驟。這改善了人類的可解釋性和可信度。 ReAct 提示的模型會為每次叠代生成思考行動觀察三元組，你很快就會看到。開始吧！


## 設定


In [ ]:
!pip install -q google.generativeai

In [ ]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


備註：[ `wikipedia` 套件](https://pypi.org/project/wikipedia/) 指出它「以簡單易用為設計，而非進階用途」，而生產或大量使用應「使用 [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) 或其他進階 [Python MediaWiki API 封裝](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)」。


In [ ]:
import re
import os

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import google.generativeai as genai

要運行下列單元格，你的 API 金鑰必須儲存在名為「GOOGLE_API_KEY」的 Colab Secret 中。如果你尚未擁有 API 金鑰，或者不確定如何建立 Colab Secret，請查看[驗證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門以取得範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## ReAct 輸入提示


論文中使用的提示可在 [https://github.com/ysymyth/ReAct/tree/master/prompts](https://github.com/ysymyth/ReAct/tree/master/prompts) 取得

你將會使用以下 ReAct 提示，並進行幾項小調整。


> 備註：此處使用的提示和情境範例取自 [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct)，其在 [麻省理工學院授權](https://opensource.org/licenses/MIT) 下發布。


In [ ]:
model_instructions = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, Observation is understanding relevant information from an Action's output and Action can be of three types:
(1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search and you can try to search the information from those topics.
(2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches, so keep your searches short.
(3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

### 少樣本提示，以 Gemini 啟用情境內學習


雖然大型語言模型對於指令提示語有良好的理解能力，但在零次拍攝設定中，在執行複雜任務時仍可能表現不佳。因此，現在你需要提供幾個範例以及你的提示語，以根據你的需求來引導模型輸出。這種在情境中的學習顯著提升了模型的效能。


In [ ]:
examples = """
Here are some examples.

Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>

Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>

Question
Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?

Thought 1
I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.

Action 1
<search>Adam Clayton Powell</search>

Observation 1
Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].

Thought 2
To find the documentary, I can search Adam Clayton Powell (film).

Action 2
<search>Adam Clayton Powell (film)</search>

Observation 2
Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.

Thought 3
Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.

Action 3
<finish>The Saimaa Gesture</finish>

Question
What profession does Nicholas Ray and Elia Kazan have in common?

Thought 1
I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.

Action 1
<search>Nicholas Ray</search>

Observation 1
Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.

Thought 2
Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.

Action 2
<search>Elia Kazan</search>

Observation 2
Elia Kazan was an American film and theatre director, producer, screenwriter and actor.

Thought 3
Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.

Action 3
<finish>director, screenwriter, actor</finish>

Question
Which magazine was started first Arthur’s Magazine or First for Women?

Thought 1
I need to search Arthur’s Magazine and First for Women, and find which was started first.

Action 1
<search>Arthur’s Magazine</search>

Observation 1
Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.

Thought 2
Arthur’s Magazine was started in 1844. I need to search First for Women next.

Action 2
<search>First for Women</search>

Observation 2
First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.

Thought 3
First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.

Action 3
<finish>Arthur’s Magazine</finish>

Question
Were Pavel Urysohn and Leonid Levin known for the same type of work?

Thought 1
I need to search Pavel Urysohn and Leonid Levin, find their types of work, then find if they are the same.

Action 1
<search>Pavel Urysohn</search>

Observation 1
Pavel Samuilovich Urysohn (February 3, 1898 - August 17, 1924) was a Soviet mathematician who is best known for his contributions in dimension theory.

Thought 2
Pavel Urysohn is a mathematician. I need to search Leonid Levin next and find its type of work.

Action 2
<search>Leonid Levin</search>

Observation 2
Leonid Anatolievich Levin is a Soviet-American mathematician and computer scientist.

Thought 3
Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.

Action 3
<finish>yes</finish>

Question
{question}"""

將附有範例的說明複製到稱為 `model_instructions.txt` 的檔案中


In [ ]:
ReAct_prompt = model_instructions + examples
with open('model_instructions.txt', 'w') as f:
  f.write(ReAct_prompt)

## Gemini-ReAct 管道


### 設定


你現在將建立一個端到端管線，以促進使用 ReAct 提示的 Gemini 模型進行多輪對話。


In [ ]:
class ReAct:
  def __init__(self, model: str, ReAct_prompt: str | os.PathLike):
    """Prepares Gemini to follow a `Few-shot ReAct prompt` by imitating
    `function calling` technique to generate both reasoning traces and
    task-specific actions in an interleaved manner.

    Args:
        model: name to the model.
        ReAct_prompt: ReAct prompt OR path to the ReAct prompt.
    """
    self.model = genai.GenerativeModel(model)
    self.chat = self.model.start_chat(history=[])
    self.should_continue_prompting = True
    self._search_history: list[str] = []
    self._search_urls: list[str] = []

    try:
      # try to read the file
      with open(ReAct_prompt, 'r') as f:
        self._prompt = f.read()
    except FileNotFoundError:
      # assume that the parameter represents prompt itself rather than path to the prompt file.
      self._prompt = ReAct_prompt

  @property
  def prompt(self):
    return self._prompt

  @classmethod
  def add_method(cls, func):
    setattr(cls, func.__name__, func)

  @staticmethod
  def clean(text: str):
    """Helper function for responses."""
    text = text.replace("\n", " ")
    return text

### 定義工具


按照提示所述，該模型將產生 **想法-動作-觀察** 追蹤，其中每個 **動作** 追蹤可能是下列程式碼之一：


1.   </search/> ：透過外部 API 進行 Wikipedia 搜尋。
2.   </lookup/> ：使用 Wikipedia API 查詢頁面上特定資訊。
3.   </finish/> ：停止執行模型並傳回答案。

如果模型遇到這些程式碼中的任何一個程式碼，模型應該利用提供給模型的 `tools`。模型對利用獲取的工具組從外部世界收集資訊的這項理解，通常稱為 **函式呼叫** 。因此，下一個目標是模擬這個函式呼叫技術，以便讓 ReAct 提示的 Gemini 模型能夠存取外部真實資訊。

Gemini API 支援函式呼叫，可以使用這個功能設定你的工具。然而，在本教學課程中，你將學習使用 `stop_sequences` 參數來模擬。


定義工具：


#### 搜尋
定義一種執行維基百科搜尋的方法


In [ ]:
@ReAct.add_method
def search(self, query: str):
    """Perfoms search on `query` via Wikipedia api and returns its summary.

    Args:
        query: Search parameter to query the Wikipedia API with.

    Returns:
        observation: Summary of Wikipedia search for `query` if found else
        similar search results.
    """
    observation = None
    query = query.strip()
    try:
      # try to get the summary for requested `query` from the Wikipedia
      observation = wikipedia.summary(query, sentences=4, auto_suggest=False)
      wiki_url = wikipedia.page(query, auto_suggest=False).url
      observation = self.clean(observation)

      # if successful, return the first 2-3 sentences from the summary as model's context
      observation = self.model.generate_content(f'Retun the first 2 or 3 \
      sentences from the following text: {observation}')
      observation = observation.text

      # keep track of the model's search history
      self._search_history.append(query)
      self._search_urls.append(wiki_url)
      print(f"Information Source: {wiki_url}")

    # if the page is ambiguous/does not exist, return similar search phrases for model's context
    except (DisambiguationError, PageError) as e:
      observation = f'Could not find ["{query}"].'
      # get a list of similar search topics
      search_results = wikipedia.search(query)
      observation += f' Similar: {search_results}. You should search for one of those instead.'

    return observation

#### Lookup
在維基百科頁面中尋找特定的詞組。


In [ ]:
@ReAct.add_method
def lookup(self, phrase: str, context_length=200):
    """Searches for the `phrase` in the lastest Wikipedia search page
    and returns number of sentences which is controlled by the
    `context_length` parameter.

    Args:
        phrase: Lookup phrase to search for within a page. Generally
        attributes to some specification of any topic.

        context_length: Number of words to consider
        while looking for the answer.

    Returns:
        result: Context related to the `phrase` within the page.
    """
    # get the last searched Wikipedia page and find `phrase` in it.
    page = wikipedia.page(self._search_history[-1], auto_suggest=False)
    page = page.content
    page = self.clean(page)
    start_index = page.find(phrase)

    # extract sentences considering the context length defined
    result = page[max(0, start_index - context_length):start_index+len(phrase)+context_length]
    print(f"Information Source: {self._search_urls[-1]}")
    return result

#### 完畢
指示管道終止執行命令。


In [ ]:
@ReAct.add_method
def finish(self, _):
  """Finishes the conversation on encountering <finish> token by
  setting the `self.should_continue_prompting` flag to `False`.
  """
  self.should_continue_prompting = False
  print(f"Information Sources: {self._search_urls}")

### 終止 Token 與函式呼叫


現在你已完成函式定義，下一步是指示模型在遇到任一個動作 Token 時中斷其執行。你將使用 [`genai.GenerativeModel.GenerationConfig`](https://ai.google.dev/api/python/google/generativeai/GenerationConfig) 課程中的 `stop_sequences` 參數，來指示模型何時停止。在遇到動作 Token 時，執行流程會從 `stop_sequences` 參數中擷取只有一個特定的 Token 設定模型的執行停止，然後呼叫適當的 **工具** (函式)。

函式的回應會新增到模型的對話記錄中，供繼續對話使用。


In [ ]:
@ReAct.add_method
def __call__(self, user_question, max_calls: int=8, **generation_kwargs):
  """Starts multi-turn conversation with the chat models with function calling

  Args:
      max_calls: max calls made to the model to get the final answer.

      generation_kwargs: Same as genai.GenerativeModel.GenerationConfig
              candidate_count: (int | None) = None,
              stop_sequences: (Iterable[str] | None) = None,
              max_output_tokens: (int | None) = None,
              temperature: (float | None) = None,
              top_p: (float | None) = None,
              top_k: (int | None) = None

  Raises:
      AssertionError: if max_calls is not between 1 and 8
  """

  # hyperparameter fine-tuned according to the paper
  assert 0 < max_calls <= 8, "max_calls must be between 1 and 8"

  if len(self.chat.history) == 0:
    model_prompt = self.prompt.format(question=user_question)
  else:
    model_prompt = user_question

  # stop_sequences for the model to immitate function calling
  callable_entities = ['</search>', '</lookup>', '</finish>']

  generation_kwargs.update({'stop_sequences': callable_entities})

  self.should_continue_prompting = True
  for idx in range(max_calls):

    self.response = self.chat.send_message(content=[model_prompt],
              generation_config=generation_kwargs, stream=False)

    for chunk in self.response:
      print(chunk.text, end=' ')

    response_cmd = self.chat.history[-1].parts[-1].text

    try:
      # regex to extract <function name writen in between angular brackets>
      cmd = re.findall(r'<(.*)>', response_cmd)[-1]
      print(f'</{cmd}>')
      # regex to extract param
      query = response_cmd.split(f'<{cmd}>')[-1].strip()
      # call to appropriate function
      observation = self.__getattribute__(cmd)(query)

      if not self.should_continue_prompting:
        break

      stream_message = f"\nObservation {idx + 1}\n{observation}"
      print(stream_message)
      # send function's output as user's response
      model_prompt = f"<{cmd}>{query}</{cmd}>'s Output: {stream_message}"

    except (IndexError, AttributeError) as e:
      model_prompt = "Please try to generate thought-action-observation traces \
      as instructed by the prompt."

### 測試 ReAct 指令的 Gemini 範例


In [ ]:
gemini_ReAct_chat = ReAct(model='gemini-pro', ReAct_prompt='model_instructions.txt')
# Note: try different combinations of generational_config parameters for variational results
gemini_ReAct_chat("What are the total of ages of the main trio from the new Percy Jackson and the Olympians TV series in real life?", temperature=0.2)

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2107.88ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1876.20ms


Action 1
<search>Percy Jackson and the Olympians TV series </search>

Observation 1
Could not find ["Percy Jackson and the Olympians TV series"]. Similar: ['Percy Jackson and the Olympians (TV series)', 'Percy Jackson & the Olympians', 'Percy Jackson (film series)', 'Percy Jackson & the Olympians: The Lightning Thief', 'Percy Jackson (disambiguation)', 'Percy Jackson', 'List of characters in mythology novels by Rick Riordan', 'The Lightning Thief', 'The Heroes of Olympus', 'Walker Scobell']. You should search for one of those instead.
Action 2
<search>Percy Jackson and the Olympians (TV series) </search>
Information Source: https://en.wikipedia.org/wiki/Percy_Jackson_and_the_Olympians_(TV_series)

Observation 2
Percy Jackson and the Olympians is an American fantasy television series created by Rick Riordan and Jonathan E. Steinberg for Disney+, based on the book series of the same name by Riordan. Walker Scobell stars as Percy Jackson, alongside Leah Sava Jeffries and Aryan Simhadri. 


ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 433.49ms


Thought 1
I need to find the ages of Walker Scobell, Leah Sava Jeffries, and Aryan Simhadri, then sum them up.

Action 3
<search>Walker Scobell </search>


ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 410.12ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 460.87ms


Information Source: https://en.wikipedia.org/wiki/Walker_Scobell

Observation 3
Walker Scobell (born January 5, 2009) is an American actor. He has starred in the 2022 action comedy films The Adam Project and Secret Headquarters. In 2023, he began playing the title character of Percy Jackson in the Disney+ fantasy series Percy Jackson and the Olympians.
Thought 2
Walker Scobell was born on January 5, 2009. I need to find the ages of Leah Sava Jeffries and Aryan Simhadri next.

Action 4
<search>Leah Sava Jeffries </search>
Information Source: https://en.wikipedia.org/wiki/Leah_Jeffries

Observation 4
Leah Sava Jeffries is an American child actress born on September 25, 2009. She made her acting debut in the American musical drama, Empire (2015) and later her feature film debut in the action-thriller Beast (2022).
Thought 3
Leah Sava Jeffries was born on September 25, 2009. I need to find the age of Aryan Simhadri next.

Action 5
<search>Aryan Simhadri </search>
Information Source: https:

現在，嘗試在沒有 ReAct 提示的情況下向 `gemini-pro` 模型詢問相同的問題。


In [ ]:
gemini_ReAct_chat.model.generate_content("What is the total of ages of the main trio from the new Percy Jackson and the Olympians TV series in real life?").text

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 584.37ms


"The main trio from the new Percy Jackson and the Olympians TV series haven't been cast yet, so their real-life ages are unknown."

## 總結

以 ReAct 提示的 Gemini 模型以外部資訊來源為基礎，因此不太容易產生幻覺。此外，此模型產生的思維-行動-觀察追蹤，能讓使用者見證模型根據使用者查詢所進行的推理過程，進而提升人類對於其可解釋性與可信度的評價。


## 下一步


前往這一個 [Streamlit 應用程式](https://mayochat.streamlit.app/) 與一個使用這個程式碼建構出的由 ReAct 提示的 Gemini 機器人互動。
